In [235]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import yfinance as yf

In [179]:
empresas = pd.read_csv('empresas.csv')
infos = pd.read_csv('infos_empresas.csv')
cotacoes = pd.read_csv('cotacoes.csv')
dividendos = pd.read_csv('dividendos.csv')

empresas = empresas.drop(empresas.columns[0], axis=1)
infos = infos.drop(infos.columns[0], axis=1)
cotacoes = cotacoes.drop(cotacoes.columns[0], axis=1)
dividendos = dividendos.drop(dividendos.columns[0], axis=1)

In [180]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\3737537357.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='bfill')


In [181]:
infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\1645137256.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  infos['value'] = infos.groupby(['cvm_code', 'data'])['value'].fillna(method='ffill')


In [182]:
anos = np.arange(2015,2023)
anos

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [183]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1


In [184]:
df_list = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    cod_cvm = empresas.iloc[i]['codigo_cvm']
    n_de_acoes = empresas.iloc[i]['acoes_total']
    setor = empresas.iloc[i]['setor']
    estatal = empresas.iloc[i]['estatal']
    
    for ano in anos:
        dados_empresa = infos[infos['cvm_code']==cod_cvm]
        dados_empresa_ano = dados_empresa[dados_empresa['year']==ano]

        try:
            lucro_liquido = dados_empresa_ano[dados_empresa_ano['data']=='net_income']['value'].values[0]
        except:
            lucro_liquido = 1
        try:
            patrimonio_liquido = dados_empresa_ano[dados_empresa_ano['data']=='equity']['value'].values[0]
        except:
            patrimonio_liquido = 1
        try:
            divida_liquida = dados_empresa_ano[dados_empresa_ano['data']=='net_debt']['value'].values[0]
        except:
            divida_liquida = 1

        roe = lucro_liquido/patrimonio_liquido
        debt_over_income = divida_liquida/lucro_liquido
        vpa = patrimonio_liquido/n_de_acoes
        lpa = lucro_liquido/n_de_acoes

        dicionario = {
            'ticker' : ticker,
            'cvm_code' : cod_cvm,
            'setor' : setor,
            'estatal' : estatal,
            'year' : ano,
            'roe' : roe,
            'debt_ratio' : debt_over_income,
            'vpa' : vpa,
            'lpa' : lpa
        }

        df = pd.DataFrame(dicionario, index=[0])
        df_list.append(df)

C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:26: RuntimeWarning: invalid value encountered in scalar divide
  roe = lucro_liquido/patrimonio_liquido
C:\Users\eduar\AppData\Local\Temp\ipykernel_19396\2447898130.py:27: RuntimeWarning: divide by zero encountered in scalar divide
  debt_over_income = divida_liquida/lucro_liquido


In [185]:
indicadores = pd.concat(df_list)
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075
...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000


In [186]:
dividendos_list = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    dividendos_ticker_ano = dividendos[dividendos['ticker']==ticker][dividendos[dividendos['ticker']==ticker]['year']==ano]
    dividendo_ano_valor = dividendos_ticker_ano['value'].sum()

    dividendos_list.append(dividendo_ano_valor)

In [187]:
indicadores['dividendo'] = dividendos_list

In [188]:
indicadores

,ticker,cvm_code,setor,estatal,year,roe,debt_ratio,vpa,lpa,dividendo
0,CMIG4,2453,energia,True,2015,0.190104,4.785397,5.899813,1.121575,1.219627
0,CMIG4,2453,energia,True,2016,0.025881,39.342204,5.875599,0.152066,1.293778
0,CMIG4,2453,energia,True,2017,0.069894,12.289442,6.509642,0.454987,0.487119
0,CMIG4,2453,energia,True,2018,0.109271,7.565812,7.240637,0.791195,1.288606
0,CMIG4,2453,energia,True,2019,0.198376,4.226188,7.314763,1.451075,1.450221
...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,24929,telecom,False,2018,0.079137,-1.500065,8.005929,0.633563,0.236894
0,TIMS3,24929,telecom,False,2019,NaN,-inf,0.000000,0.000000,0.465888
0,TIMS3,24929,telecom,False,2020,0.079529,-1.247876,9.576464,0.761602,0.549765
0,TIMS3,24929,telecom,False,2021,NaN,inf,0.000000,0.000000,0.432831


In [190]:
equivalencia_empresas_ticker_list = []
equivalencia_empresas_valor_list = []
for cvm_code in empresas['codigo_cvm'].unique():
    ticker_list = empresas[empresas['codigo_cvm']==cvm_code]['empresa'].values
    ticker_principal = ticker_list[0]
    
    if len(ticker_list)==1:
        equivalencia_empresas_ticker_list.append(ticker_principal)
        equivalencia_empresas_valor_list.append(1)
    else:
        for ticker in ticker_list:
            dividendos_ticker = indicadores[indicadores['ticker']==ticker]['dividendo'].mean()
            dividendos_ticker_principal = indicadores[indicadores['ticker']==ticker_principal]['dividendo'].mean()
            multiplicador = dividendos_ticker/dividendos_ticker_principal

            equivalencia_empresas_ticker_list.append(ticker_principal)
            equivalencia_empresas_valor_list.append(multiplicador)


In [191]:
empresas['equivalencia_ticker'] = equivalencia_empresas_ticker_list
empresas['equivalencia_valor'] = equivalencia_empresas_valor_list

In [192]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit,equivalencia_ticker,equivalencia_valor
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,1.000000
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,0.924145
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3,TAEE11,1.000000
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,1.000000
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,0.908667
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1,CSMG3,1.000000
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5,SAPR11,1.000000
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1,SAPR11,0.290275


In [193]:
multiplos = {
    'energia': {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'saneamento' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7},
    'banco' : {'vpa':1.4,
                'roe': 0.15,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.6},
    'seguro' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 12,
                'dy': 8,
                'estatal_mult': 0.7},
    'telecom' : {'vpa':1.0,
                'roe': 0.10,
                'pl': 10,
                'dy': 8,
                'estatal_mult': 0.7},
    'combustivel' : {'vpa':1.2,
                'roe': 0.15,
                'pl': 7,
                'dy': 8,
                'estatal_mult': 0.7}
}

In [ ]:
df_list_indicadores_val = []
for i in range(len(indicadores)):
    ticker = indicadores.iloc[i]['ticker']
    ano = indicadores.iloc[i]['year']

    cotacoes_empresa = cotacoes[cotacoes['ticker']==ticker]
    cotacoes_empresa_ano = cotacoes_empresa[cotacoes_empresa['year']==ano]

    try:
        pl_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['lpa']
        pl_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['lpa']
        pl_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['lpa']
    except:
        pl_max = 0
        pl_min = 0
        pl_clo = 0

    try:
        pvp_max = cotacoes_empresa_ano['max'].values[0]/indicadores.iloc[i]['vpa']
        pvp_min = cotacoes_empresa_ano['min'].values[0]/indicadores.iloc[i]['vpa']
        pvp_clo = cotacoes_empresa_ano['close'].values[0]/indicadores.iloc[i]['vpa']
    except:
        pvp_max = 0
        pvp_min = 0
        pvp_clo = 0

    try:
        dy_max = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['max'].values[0]
        dy_min = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['min'].values[0]
        dy_clo = indicadores.iloc[i]['dividendo']/cotacoes_empresa_ano['close'].values[0]
    except:
        dy_max = 0
        dy_min = 0
        dy_clo = 0

    dicionario = {
        'ticker': ticker,
        'year': ano,

        'pl':pl_clo,
        'pl_max': pl_max,
        'pl_min': pl_min,

        'pvp':pvp_clo,
        'pvp_max': pvp_max,
        'pvp_min': pvp_min,

        'dy':dy_clo,
        'dy_max': dy_max,
        'dy_min': dy_min
    }

    indicadores_values = pd.DataFrame(dicionario, index=[0])
    df_list_indicadores_val.append(indicadores_values)

In [ ]:
faixas_indicadores = pd.concat(df_list_indicadores_val)

In [230]:
faixas_indicadores

,ticker,year,pl,pl_max,pl_min,pvp,pvp_max,pvp_min,dy,dy_max,dy_min
0,CMIG4,2015,5.465527,14.336978,5.135634,1.039016,2.725510,0.976302,0.198960,0.075847,0.211741
0,CMIG4,2016,50.701614,65.563565,26.961948,1.312207,1.696848,0.697801,0.167805,0.129767,0.315556
0,CMIG4,2017,15.099318,26.462269,11.297015,1.055358,1.849564,0.789598,0.070905,0.040458,0.094770
0,CMIG4,2018,17.517810,17.530449,8.164867,1.914196,1.915577,0.892187,0.092973,0.092906,0.199475
0,CMIG4,2019,9.503302,10.847134,8.407562,1.885229,2.151813,1.667860,0.105165,0.092136,0.118871
...,...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2020,19.235763,19.879144,14.994704,1.529792,1.580959,1.192507,0.037527,0.036312,0.048141
0,TIMS3,2021,inf,inf,inf,inf,inf,inf,0.032915,0.029147,0.038819


In [240]:
cotacao_list = []
for ticker in empresas.empresa:
    ticker = ticker + '.SA'

    acao = yf.Ticker(ticker)

    try:
        cotação_atualizada = acao.history(period="1d")["Close"].iloc[-1]
    except:
        cotação_atualizada = 0

    cotacao_list.append(cotação_atualizada)

BRSR5.SA: No price data found, symbol may be delisted (period=1d)


In [241]:
empresas['cotacao'] = cotacao_list

In [242]:
empresas

,empresa,setor,codigo_cvm,estatal,acoes_on,acoes_pn,acoes_total,equivalencia_unit,equivalencia_ticker,equivalencia_valor,cotacao
0,CMIG4,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,1.000000,11.820000
1,CMIG3,energia,2453,True,735847624,1465523064,2201370688,1,CMIG4,0.924145,14.520000
2,TAEE11,energia,20257,False,590714069,442782652,1033496721,3,TAEE11,1.000000,35.439999
3,TAEE3,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737,11.690000
4,TAEE4,energia,20257,False,590714069,442782652,1033496721,1,TAEE11,0.337737,11.880000
5,CPLE6,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,1.000000,10.040000
6,CPLE3,energia,14311,True,1300347300,1682463291,2982810591,1,CPLE6,0.908667,9.040000
7,CSMG3,saneamento,19445,True,380253069,0,380253069,1,CSMG3,1.000000,20.190001
8,SAPR11,saneamento,18627,True,503735259,1007470260,1511205519,5,SAPR11,1.000000,25.469999
9,SAPR3,saneamento,18627,True,503735259,1007470260,1511205519,1,SAPR11,0.290275,4.990000


In [243]:
faixas_indicadores

,ticker,year,pl,pl_max,pl_min,pvp,pvp_max,pvp_min,dy,dy_max,dy_min
0,CMIG4,2015,5.465527,14.336978,5.135634,1.039016,2.725510,0.976302,0.198960,0.075847,0.211741
0,CMIG4,2016,50.701614,65.563565,26.961948,1.312207,1.696848,0.697801,0.167805,0.129767,0.315556
0,CMIG4,2017,15.099318,26.462269,11.297015,1.055358,1.849564,0.789598,0.070905,0.040458,0.094770
0,CMIG4,2018,17.517810,17.530449,8.164867,1.914196,1.915577,0.892187,0.092973,0.092906,0.199475
0,CMIG4,2019,9.503302,10.847134,8.407562,1.885229,2.151813,1.667860,0.105165,0.092136,0.118871
...,...,...,...,...,...,...,...,...,...,...,...
0,TIMS3,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,TIMS3,2020,19.235763,19.879144,14.994704,1.529792,1.580959,1.192507,0.037527,0.036312,0.048141
0,TIMS3,2021,inf,inf,inf,inf,inf,inf,0.032915,0.029147,0.038819


In [249]:
df_list_precos = []
for i in range(len(empresas)):
    ticker = empresas.iloc[i]['empresa']
    estatal = empresas.iloc[i]['estatal']
    equivalencia_unit = empresas.iloc[i]['equivalencia_unit']
    cotacao = empresas.iloc[i]['cotacao']

    dividendos_por_acao_total = indicadores[indicadores['ticker']==ticker]['dividendo'].mean()
    dividendos_por_acao_3_dados = indicadores[indicadores['ticker']==ticker]['dividendo'].iloc[-3:].mean()
    dividendos_por_acao_5_dados = indicadores[indicadores['ticker']==ticker]['dividendo'].iloc[-5:].mean()

    roe = indicadores[indicadores['ticker']==ticker]['roe'].iloc[-1]

    ultimo_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].iloc[-1]
    media_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].mean()
    media_ultimos_3_lpa = indicadores[indicadores['ticker']==ticker]['lpa'].iloc[-3:].mean()

    media_pl_min = faixas_indicadores[faixas_indicadores['ticker']==ticker]['pl_min'].iloc[-3:].mean()

    ultimo_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].iloc[-1]
    media_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].mean()
    media_ultimos_3_vpa = indicadores[indicadores['ticker']==ticker]['vpa'].iloc[-3:].mean()

    media_pvp_min = faixas_indicadores[faixas_indicadores['ticker']==ticker]['pvp_min'].iloc[-3:].mean()

    indice_pvp = multiplos[setor]['vpa']
    if estatal:
        indice_pvp = indice_pvp * multiplos[setor]['estatal_mult']

    indice_pl = multiplos[setor]['pl']
    if estatal:
        indice_pl = indice_pl * multiplos[setor]['estatal_mult']

    indice_div = multiplos[setor]['dy']
    indice_div = indice_div/100

    preco_teto_3_8 = dividendos_por_acao_3_dados/indice_div
    if media_ultimos_3_vpa > preco_teto_3_8:
        preco_teto_3_8_ajustato = (media_ultimos_3_vpa - preco_teto_3_8)/2 + preco_teto_3_8
    else:
        preco_teto_3_8_ajustato = preco_teto_3_8

    preco_teto_vpa = media_ultimos_3_vpa * indice_pvp * equivalencia_unit
    if cotacao > preco_teto_vpa*1.5 or preco_teto_vpa > cotacao * 0.5:
        preco_teto_vpa_ajustado = media_pvp_min * cotacao
    else:
        preco_teto_vpa_ajustado = preco_teto_vpa

    preco_teto_lpa = media_ultimos_3_lpa * indice_pl * equivalencia_unit
    if cotacao > preco_teto_lpa*1.5 or preco_teto_lpa > cotacao * 0.5:
        preco_teto_lpa_ajustado = media_pl_min * cotacao
    else:
        preco_teto_lpa_ajustado = preco_teto_lpa


    dicionario={
        'ticker': ticker,
        'estatal': estatal,
        'equivalencia_unit': equivalencia_unit,
        'div_pa': dividendos_por_acao_total,
        'div3_pa': dividendos_por_acao_3_dados,
        'lpa': ultimo_lpa,
        'l3pa': media_ultimos_3_lpa,
        'vpa': ultimo_vpa,
        'v3pa': media_ultimos_3_vpa,
        'indice_pl': indice_pl,
        'indice_pvp': indice_pvp,
        'indice_div': indice_div,
        'preco_3_8': preco_teto_3_8,
        'preco_3_8_ajustado': preco_teto_3_8_ajustato,
        'preco_teto_vpa': preco_teto_vpa,
        'preco_teto_vpa_ajustado': preco_teto_vpa_ajustado,
        'preco_teto_lpa': preco_teto_lpa,
        'preco_teto_lpa_ajustado': preco_teto_lpa_ajustado,
    }

    df=pd.DataFrame(dicionario,index=[0])
    
    df_list_precos.append(df) 


In [250]:
preco_teto = pd.concat(df_list_precos)

In [251]:
preco_teto

,ticker,estatal,equivalencia_unit,div_pa,div3_pa,lpa,l3pa,vpa,v3pa,indice_pl,indice_pvp,indice_div,preco_3_8,preco_3_8_ajustado,preco_teto_vpa,preco_teto_vpa_ajustado,preco_teto_lpa,preco_teto_lpa_ajustado
0,CMIG4,True,1,1.548077,2.215089,1.859917,1.622074,9.895319,8.891790,7.0,0.7,0.08,27.688617,27.688617,6.224253,12.642040,11.354516,69.512530
0,CMIG3,True,1,1.430648,2.215089,1.859917,1.622074,9.895319,8.891790,7.0,0.7,0.08,27.688617,27.688617,6.224253,17.948828,11.354516,98.071565
0,TAEE11,False,3,8.851015,12.569321,1.402244,1.911264,6.357520,6.218738,10.0,1.0,0.08,157.116509,157.116509,18.656214,163.498491,57.337927,567.832016
0,TAEE3,False,1,2.989313,4.189774,1.402244,1.911264,6.357520,6.218738,10.0,1.0,0.08,52.372170,52.372170,6.218738,17.954259,19.112642,62.322261
0,TAEE4,False,1,2.989313,4.189774,1.402244,1.911264,6.357520,6.218738,10.0,1.0,0.08,52.372170,52.372170,6.218738,18.327505,19.112642,63.682419
0,CPLE6,True,1,3.612192,3.530220,0.385315,1.129547,7.084334,7.102583,7.0,0.7,0.08,44.127756,44.127756,4.971808,26.230684,7.906828,171.388372
0,CPLE3,True,1,3.282280,3.209287,0.385315,1.129547,7.084334,7.102583,7.0,0.7,0.08,40.116085,40.116085,4.971808,24.325699,7.906828,155.564927
0,CSMG3,True,1,2.287250,3.230581,2.217897,1.926284,19.078121,17.945140,7.0,0.7,0.08,40.382262,40.382262,12.561598,14.445356,13.483987,138.783478
0,SAPR11,True,5,4.815997,3.593234,0.762000,0.733523,5.814488,5.195938,7.0,0.7,0.08,44.915427,44.915427,18.185784,91.156606,25.673305,641.388976
0,SAPR3,True,1,1.397963,0.665414,0.762000,0.733523,5.814488,5.195938,7.0,0.7,0.08,8.317672,8.317672,3.637157,3.501911,5.134661,24.620124
